In [7]:
import sys
sys.path.append('/mnt/')
import numpy as np
import pandas as pd
from finrl.models.env import StockTradingEnv
from finrl.models.utils import data_split
from finrl.models.metrics import *
from finrl.models.constants import *
from finrl.data.generate import stock_trade_data_generate  # 数据生成
from finrl.models.baselines.statistics import *
# import finrl.models.baselines.statistics as statistics   # 将statistics当作包导入，前提是具有init文件。
# statistics.BestCRP  # init文件下导入的包

In [4]:
data_dir = '/mnt/finrl/data/csv/DOW_30/'
vix_data_dir = '/mnt/finrl/data/csv/^VIX/'
dji_dir = '/mnt/finrl/data/csv/DJI/DJI.csv'  # .csv

TRAIN_START_DATE = '2010-01-01'
TRAIN_END_DATE = '2021-12-31'
VALIDATE_START_DATE = '2022-01-01'
VALIDATE_END_DATE = '2022-12-31'
TEST_START_DATE = '2023-01-01'
TEST_END_DATE = '2024-01-30'

df = stock_trade_data_generate(
    data_dir=data_dir,
    start_date=TRAIN_START_DATE,
    end_date=TEST_END_DATE,
    use_technical_indicator=True,
    use_turbulence=True,
    user_defined_feature=False,
    tech_indicator_list=INDICATORS,
    use_vix=True,
    vix_data_dir=vix_data_dir,
    dji_dir=dji_dir,
)
df_train = data_split(df,TRAIN_START_DATE,TRAIN_END_DATE)
df_validation = data_split(df,VALIDATE_START_DATE,VALIDATE_END_DATE)
df_test = data_split(df,TEST_START_DATE,TEST_END_DATE)
df_train_validation = data_split(df,TRAIN_START_DATE,VALIDATE_END_DATE)

Shape of DataFrame:  (103943, 8)
Successfully added technical indicators
Shape of DataFrame:  (3542, 8)
Successfully added vix
Successfully added turbulence index


In [10]:
env_trade_kwargs = {
    'stock_dim': None,
    'hmax': 100,
    'initial_amount': 1000000,
    'num_stock_shares': None,    # [0]*stock_dim
    'buy_cost_pct': None,    # [0.001]*ACTION_DIM
    'sell_cost_pct': None,      # [0.001]*ACTION_DIM
    'reward_scaling': 1e-4,         # reward=reward*reward_scaling
    'tech_indicator_list': ['macd','boll_ub','boll_lb','rsi_30','cci_30','dx_30','close_30_sma','close_60_sma'],
}

In [13]:
# 这样更通用一点
ACTION_DIM = len(df_test.tic.unique())
STATE_DIM = 1 + ACTION_DIM * (1 + 1 + len(INDICATORS))
env_trade_kwargs.update({
    'stock_dim':ACTION_DIM,
    'statel_dim':STATE_DIM,
    'num_stock_shares':[0]*ACTION_DIM,
    'buy_cost_pct':[0.001]*ACTION_DIM,
    'sell_cost_pct':[0.001]*ACTION_DIM,
    'turbulence_threshold':None,  #70  可以设置预警
    'cash_norm_factor':1.0,
    'num_share_norm_factor':1,  # 持股数不归一化
    'risk_indicator_col':'vix'
    })
env_gym_trade = StockTradingEnv(df_test,**env_trade_kwargs)
print(env_trade_kwargs)

{'stock_dim': 29, 'hmax': 100, 'initial_amount': 1000000, 'num_stock_shares': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'buy_cost_pct': [0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001], 'sell_cost_pct': [0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001], 'reward_scaling': 0.0001, 'tech_indicator_list': ['macd', 'boll_ub', 'boll_lb', 'rsi_30', 'cci_30', 'dx_30', 'close_30_sma', 'close_60_sma'], 'turbulence_threshold': None, 'risk_indicator_col': 'vix', 'cash_norm_factor': 1.0, 'num_share_norm_factor': 1}


# MACD


## 测试

获得交易动作

In [14]:
MACDBaseline=MACDStrategy()
position_sizes = pd.DataFrame()
for tic in df_test.tic.unique():
    s=df_test[df_test.tic==tic].close  # 提取对应股票数据
    s.index=df_test[df_test.tic==tic].date
    signals = pd.Series(MACDBaseline.calc_combined_signal(s))
    signals.index = s.index
    position_size = MACDBaseline.scale_signal(signals)  
    position_sizes[tic]=position_size

和环境交互

In [15]:
done=False
i=0
while not done:
    a=position_sizes.iloc[i]
    a=np.array(a)
    s,r,done,_=env_gym_trade.step(a)
    i+=1

计算测试结果

In [16]:
# account_value = pd.DataFrame(env_gym_trade.asset_memory)
# account_value.columns=['account_value']
# account_value.insert(0,'date',df_test.date.unique())
# backtest_stats(account_value)

print(f'Sharpe Ratio: {sharpe_ratio(env_gym_trade.returns)}')
print(f'Cumulative Return: {cum_returns_final(env_gym_trade.returns)}')
print(f'Annual Return: {annual_return(env_gym_trade.returns)}')
print(f'Max Drawdown{max_drawdown(env_gym_trade.returns)}')

Sharpe Ratio: 1.7181
Cumulative Return: 0.3339
Annual Return: 0.3098
Max Drawdown-0.1193


# ImmediateStrategy

In [18]:
ImmediateStrategyBaseline = ImmediateStrategy(0)   #w=0即为sgn
position_sizes = pd.DataFrame()
for tic in df_test.tic.unique():
    s=df_test[df_test.tic==tic].close
    s.index=df_test[df_test.tic==tic].date
    signals = pd.Series(ImmediateStrategyBaseline.calc_trend_intermediate_signal(s))
    signals.index = s.index
    position_size = ImmediateStrategyBaseline.scale_signal(signals)
    position_sizes[tic]=position_size

In [19]:
done=False
i=0
env_gym_trade.reset()
while not done:
    a=position_sizes.iloc[i]
    a=np.array(a)
    s,r,done,_=env_gym_trade.step(a)
    i+=1

In [20]:
# account_value = pd.DataFrame(env_gym_trade.asset_memory)
# account_value.columns=['account_value']
# account_value.insert(0,'date',df_test.date.unique())
# backtest_stats(account_value)

print(f'Sharpe Ratio: {sharpe_ratio(env_gym_trade.returns)}')
print(f'Cumulative Return: {cum_returns_final(env_gym_trade.returns)}')
print(f'Annual Return: {annual_return(env_gym_trade.returns)}')
print(f'Max Drawdown{max_drawdown(env_gym_trade.returns)}')

Sharpe Ratio: 1.2382
Cumulative Return: 0.1993
Annual Return: 0.1856
Max Drawdown-0.1161


# LongOnly

In [21]:
LongOnly = LongOnlyStrategy(ACTION_DIM)

In [22]:
done=False
env_gym_trade.reset()
while not done:
    a=LongOnly.cal_signal()
    a=np.array(a)
    s,r,done,_=env_gym_trade.step(a)

In [23]:
# account_value = pd.DataFrame(env_gym_trade.asset_memory)
# account_value.columns=['account_value']
# account_value.insert(0,'date',df_test.date.unique())
# backtest_stats(account_value)

print(f'Sharpe Ratio: {sharpe_ratio(env_gym_trade.returns)}')
print(f'Cumulative Return: {cum_returns_final(env_gym_trade.returns)}')
print(f'Annual Return: {annual_return(env_gym_trade.returns)}')
print(f'Max Drawdown{max_drawdown(env_gym_trade.returns)}')

Sharpe Ratio: 1.4813
Cumulative Return: 0.1831
Annual Return: 0.1706
Max Drawdown-0.0817
